In [1]:
import numpy as np
import brevitas.nn as qnn
from brevitas.core.quant import QuantType
from torch.nn import Module
from torch import nn
from torch import Tensor
from torch import from_numpy
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from brevitas.quant_tensor import pack_quant_tensor

In [31]:
class XorDataset(Dataset):
    def __init__(self):
        a = Tensor([0,0])
        b = Tensor([0,1])
        c = Tensor([1,0])
        d = Tensor([1,1])
        aq = pack_quant_tensor(a,1.0,1.0)
        bq = pack_quant_tensor(b,1.0,1.0)
        cq = pack_quant_tensor(c,1.0,1.0)
        dq = pack_quant_tensor(d,1.0,1.0)
        self.data=[aq,bq,cq,dq]
        self.key=[0.,1.,1.,0.]

    def __getitem__(self, index):
        x = self.data[index]
        y = self.key[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [32]:
class QuantXORNet(Module):
    def __init__(self):
        super(QuantXORNet, self).__init__()
        #self.batch1 = qnn.QuantBatchNorm2d(num_features=n_inputs, 
        #                                  momentum=0.6,
        #                                  bias_quant_type=QuantType.INT)
        self.relu0 = qnn.QuantReLU(quant_type=QuantType.INT,
                                  bit_width=4.0,
                                  max_val=8)
        self.linear1 = qnn.QuantLinear(in_features = 2, 
                                       out_features=2, 
                                       bias_quant_type=QuantType.INT,
                                       bias=True,
                                       compute_output_scale=True,
                                       compute_output_bit_width=True,
                                       #input_bit_width=32,
                                       weight_quant_type=QuantType.INT)
        self.relu1 = qnn.QuantReLU(quant_type=QuantType.INT,
                                  bit_width=4,
                                  max_val=8)
        #self.batch2 = qnn.QuantBatchNorm2d(num_features=31, 
        #                                   momentum=0.6,
        #                                   bias_quant_type=QuantType.FP)
        self.linear2 = qnn.QuantLinear(in_features = 2, 
                                       out_features=2, 
                                       bias_quant_type=QuantType.INT, 
                                       bias=True,
                                       compute_output_scale=True,
                                       compute_output_bit_width=True,
                                       #bit_width=4,
                                       weight_quant_type=QuantType.INT)
        self.relu2 = qnn.QuantReLU(quant_type=QuantType.INT,
                                  bit_width=4,
                                  max_val=8)
        #self.batch3 = qnn.QuantBatchNorm2d(num_features=11, 
        #                                   momentum=0.6,
        #                                   bias_quant_type=QuantType.FP)
        self.linear3 = qnn.QuantLinear(in_features = 2, 
                                       out_features=1, 
                                       bias_quant_type=QuantType.INT, 
                                       bias=True,
                                       compute_output_scale=True,
                                       compute_output_bit_width=True,
                                       #bit_width=4,
                                       weight_quant_type=QuantType.INT)
    def forward(self, x):
        res=x
        #res = self.batch1(res)
        #res = self.relu0(res)
        #print(res)
        res = self.linear1(res)
        res = self.relu1(res)
        #res = self.relu1(self.linear1(res))
        #res = self.batch2(res)
        res = self.relu2(self.linear2(res))
        #res = self.batch3(res)
        #res = self.relu3(self.linear3(res))
        #res = self.batch4(res)
        res = self.linear3(res)
        return res

In [33]:
model = QuantXORNet()

In [34]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
criterion = nn.MSELoss()
loader = DataLoader(XorDataset())

for epoch in range(2):  # loop over the dataset multiple times
    model.train()
    criterion.train()
    running_loss = 0.0
    for i, data in enumerate(loader):
        # zero the parameter gradients
        optimizer.zero_grad()
        inputs,target = data

        # forward + backward + optimize
        print(inputs)
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

QuantTensor(tensor=tensor([[0., 0.]]), scale=tensor([1.], dtype=torch.float64), bit_width=tensor([1.], dtype=torch.float64))


RuntimeError: expected backend CPU and dtype Double but got backend CPU and dtype Float